# 加载训练模型实验

## 数据准备与加载

In [1]:
import joblib
import math
import copy
import numpy as np
import pandas as pd
from tensorflow.keras.models import load_model 
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, explained_variance_score, r2_score, mean_squared_error

In [2]:
# 加载训练数据
def load_data(path):
    df = pd.read_csv(path)
    dv = df.values
    X = dv[:, 1:-2]
    Y = dv[:, -2:]
    return X[:, :, np.newaxis], Y

# 数据准备
X02, y02 = load_data('./dataset/location_data02.csv')
X02 = np.reshape(X02, (20, 100))

# 数据再处理
X, y = load_data('./dataset/location_data.csv')
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2021)

# 针对b2_gru处理数据
X_train_2 = copy.deepcopy(X_train)
X_test_2 = copy.deepcopy(X_test)
for i in range(len(X_train_2)):
    for j in range(len(X_train_2[i])-1, 0, -1):
        X_train_2[i][j] = X_train_2[i][j] - X_train_2[i][j-1]
for i in range(len(X_test_2)):
    for j in range(len(X_test_2[i]) - 1, 0, -1):
        X_test_2[i][j] = X_test_2[i][j] - X_test_2[i][j-1]

## 导入双SVR模型计算指标

In [3]:
# 加载坐标模型
model_x = joblib.load('./models/SVR_x_2021.model')
model_y = joblib.load('./models/SVR_y_2021.model')

/Users/frank/anaconda3/envs/tensorflow/lib/python3.7/site-packages/sklearn/base.py:333: UserWarning: Trying to unpickle estimator SVR from version 0.23.2 when using version 1.0.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  UserWarning,


In [4]:
%%time
# 预测结果分析
px = model_x.predict(X02)
py = model_y.predict(X02)
pxy = [[xi, yi] for xi, yi in zip(px, py)]
print(pxy[:10])
print('MAE指标:', mean_absolute_error(y02, pxy))
print('MSE指标:', mean_squared_error(y02, pxy))
print('RMSE指标:', math.sqrt(mean_squared_error(y02, pxy)))
print('r2指标:', r2_score(y02, pxy))
print('ev指标:', explained_variance_score(y02, pxy))

[[2.019872536804463, 1.1504658854707914], [1.9174415171418377, 4.857493601728745], [2.8777523461834495, 8.218324483304807], [4.3622108033891775, 0.6274118267515414], [4.055571006636194, 6.1590862433955], [5.350674626615138, 2.8127786337651046], [6.42470271360842, 8.819694579139078], [6.980132678366614, 5.587903226497848], [8.356732356614094, 1.3613988644955959], [9.286397538342719, 7.910289796490444]]
MAE指标: 0.3727123714581049
MSE指标: 0.21134940529974042
RMSE指标: 0.45972753376292397
r2指标: 0.9740350753248724
ev指标: 0.9785745183118661
CPU times: user 17.5 ms, sys: 3.22 ms, total: 20.7 ms
Wall time: 21 ms


## 导入LSTM模型计算指标

In [5]:
# 加载预训练模型
lstm = load_model('./models/LSTM_2021.h5')
lstm.summary()

2021-12-23 15:19:46.161681: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn (SimpleRNN)       (None, 100)               10200     
_________________________________________________________________
dense (Dense)                (None, 2)                 202       
Total params: 10,402
Trainable params: 10,402
Non-trainable params: 0
_________________________________________________________________


In [6]:
%%time
# 预测测试集计算指标
pxy = lstm.predict(X_test)
pxy = np.squeeze(pxy)
print('MAE指标:', mean_absolute_error(y_test, pxy))
print('MSE指标:', mean_squared_error(y_test, pxy))
print('RMSE指标:', math.sqrt(mean_squared_error(y_test, pxy)))
print('r2指标:', r2_score(y_test, pxy))
print('ev指标:', explained_variance_score(y_test, pxy))

2021-12-23 15:19:46.644501: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)


MAE指标: 1.7231602009743456
MSE指标: 4.335757808308867
RMSE指标: 2.0822482580876054
r2指标: 0.41324465066506944
ev指标: 0.42018764615598747
CPU times: user 387 ms, sys: 64.6 ms, total: 451 ms
Wall time: 505 ms


In [7]:
%%time
# 02数据预测计算指标
X02 = np.reshape(X02, (20, 100, 1))
pxy = lstm.predict(X02)
pxy = np.squeeze(pxy)
print('MAE指标:', mean_absolute_error(y02, pxy))
print('MSE指标:', mean_squared_error(y02, pxy))
print('RMSE指标:', math.sqrt(mean_squared_error(y02, pxy)))
print('r2指标:', r2_score(y02, pxy))
print('ev指标:', explained_variance_score(y02, pxy))

MAE指标: 1.81492606818676
MSE指标: 4.801732356950433
RMSE指标: 2.191285548930224
r2指标: 0.4072556089766538
ev指标: 0.42161997006911445
CPU times: user 48.3 ms, sys: 4.72 ms, total: 53 ms
Wall time: 44.2 ms


## 导入RNN模型计算指标

In [8]:
# 加载预训练模型
rnn = load_model('./models/RNN_2021.h5')
rnn.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn (SimpleRNN)       (None, 100)               10200     
_________________________________________________________________
dense (Dense)                (None, 2)                 202       
Total params: 10,402
Trainable params: 10,402
Non-trainable params: 0
_________________________________________________________________


In [9]:
%%time
# 预测测试集计算指标
pxy = rnn.predict(X_test)
pxy = np.squeeze(pxy)
print('MAE指标:', mean_absolute_error(y_test, pxy))
print('MSE指标:', mean_squared_error(y_test, pxy))
print('RMSE指标:', math.sqrt(mean_squared_error(y_test, pxy)))
print('r2指标:', r2_score(y_test, pxy))
print('ev指标:', explained_variance_score(y_test, pxy))

MAE指标: 1.1781973842971025
MSE指标: 2.403211107677642
RMSE指标: 1.5502293726018876
r2指标: 0.6717305639104325
ev指标: 0.6726321851389807
CPU times: user 324 ms, sys: 31.9 ms, total: 356 ms
Wall time: 237 ms


In [10]:
%%time
# 02数据预测计算指标
X02 = np.reshape(X02, (20, 100, 1))
pxy = rnn.predict(X02)
pxy = np.squeeze(pxy)
print('MAE指标:', mean_absolute_error(y02, pxy))
print('MSE指标:', mean_squared_error(y02, pxy))
print('RMSE指标:', math.sqrt(mean_squared_error(y02, pxy)))
print('r2指标:', r2_score(y02, pxy))
print('ev指标:', explained_variance_score(y02, pxy))

MAE指标: 1.2487183418869972
MSE指标: 2.6452814794807122
RMSE指标: 1.6264321318397248
r2指标: 0.6743408010910564
ev指标: 0.676285087741481
CPU times: user 49.1 ms, sys: 4.23 ms, total: 53.3 ms
Wall time: 47.1 ms


## 导入DNN模型计算指标

In [11]:
# 加载预训练模型
dnn = load_model('./models/DNN_2021.h5')
dnn.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_4 (Dense)              (None, 10)                1010      
_________________________________________________________________
dense_5 (Dense)              (None, 2)                 22        
Total params: 11,132
Trainable params: 11,132
Non-trainable params: 0
_________________________________________________________________


In [12]:
%%time
# 预测测试集计算指标
X_test = np.reshape(X_test, newshape=(400, 100))
pxy = dnn.predict(X_test)
pxy = np.squeeze(pxy)
print('MAE指标:', mean_absolute_error(y_test, pxy))
print('MSE指标:', mean_squared_error(y_test, pxy))
print('RMSE指标:', math.sqrt(mean_squared_error(y_test, pxy)))
print('r2指标:', r2_score(y_test, pxy))
print('ev指标:', explained_variance_score(y_test, pxy))

MAE指标: 0.5923504185073079
MSE指标: 0.5842110479024896
RMSE指标: 0.7643369989098326
r2指标: 0.9210565716676742
ev指标: 0.9299811005363616
CPU times: user 81.7 ms, sys: 5.13 ms, total: 86.8 ms
Wall time: 92.4 ms


In [13]:
%%time
# 02数据预测计算指标
X02 = np.reshape(X02, (20, 100))
pxy = dnn.predict(X02)
pxy = np.squeeze(pxy)
print('MAE指标:', mean_absolute_error(y02, pxy))
print('MSE指标:', mean_squared_error(y02, pxy))
print('RMSE指标:', math.sqrt(mean_squared_error(y02, pxy)))
print('r2指标:', r2_score(y02, pxy))
print('ev指标:', explained_variance_score(y02, pxy))

MAE指标: 0.4972196419835091
MSE指标: 0.4285942016590419
RMSE指标: 0.6546710637098924
r2指标: 0.9472487192311967
ev指标: 0.9538703052119051
CPU times: user 42.6 ms, sys: 2.78 ms, total: 45.4 ms
Wall time: 49 ms
